# Diagnóstico Médico com Racicínio baseado em Casos (RBC)

## Resumo

O sistema foi desenvolvido utilizando a biblioteca `cbrkit` e uma [base de dados do kaggle](https://www.kaggle.com/datasets/colewelkins/cardiovascular-disease) com foco na predição de presença ou ausência de doenças cardiovasculares a partir de diversas métricas dos pacientes. O RBC foi usado com o intuito de possibilitar a comparação por similiridade de novos casos com anteriores, a fim de se ter uma base para o diagnóstico. Posteriormente, o caso novo poderia ser revisado e retido na base novamente, seguindo o fluxo dos 4Rs

<center>
<img src="src/cbr.png" width=50%>
</center>


> Para fins de eficiência, a base foi reduzida para 10.000 casos,
> bem como algumas colunas desnecessárias foram excluídas

In [65]:
# Definição das bibliotecas python utilizadas
import cbrkit
import pandas as pd
import numpy as np

Definição das variáveis na descrição da base de dados:


- **age**: Age of the patient in years (derived from age).
- **ap_hi**: Systolic blood pressure.
- **ap_lo**: Diastolic blood pressure.
- **cholesterol**: Cholesterol levels. Categorical variable (1: Normal, 2: Above Normal, 3: Well Above Normal).
- **gluc**: Glucose levels. Categorical variable (1: Normal, 2: Above Normal, 3: Well Above Normal).
- **smoke**: Smoking status. Binary variable (0: Non-smoker, 1: Smoker).
- **alco**: Alcohol intake. Binary variable (0: Does not consume alcohol, 1: Consumes alcohol).
- **active**: Physical activity. Binary variable (0: Not physically active, 1: Physically active).
- **cardio**: Presence or absence of cardiovascular disease. Target variable. Binary (0: Absence, 1: Presence).
- **bmi**: Body Mass Index, derived from weight and height. Calculated as ( $BMI = \frac{\text{weight (kg)}}{\text{height (m)}^2}$ ).
- **bp_category**: Blood pressure category based on ap_hi and ap_lo. Categories include "Normal", "Elevated", "Hypertension Stage 1", "Hypertension Stage 2", and "Hypertensive Crisis".

### Análise e Correção dos Dados

In [195]:
df = pd.read_csv("src/cardio_data_processed.csv").sample(n=10000, random_state=123)
df.loc[:, "age"] = df.loc[:, "age_years"]
df.reset_index(inplace=True)
df.drop(columns=["index", "age_years", "gender", "id", 
                 "weight", "height", "bp_category_encoded"], inplace=True)
df.head()

,age,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi,bp_category
0,43,110,70,1,1,0,0,1,0,23.388687,Normal
1,51,120,80,1,1,0,0,1,1,26.573129,Hypertension Stage 1
2,64,140,80,1,1,0,0,0,1,34.049031,Hypertension Stage 1
3,55,120,80,1,1,0,0,1,0,17.065557,Hypertension Stage 1
4,51,130,70,2,1,0,1,1,1,31.231410,Hypertension Stage 1


In [196]:
df.describe()

,age,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,52.812700,126.296800,81.239800,1.362700,1.229900,0.082900,0.047800,0.802700,0.489400,27.562413
std,6.741629,16.027102,9.273182,0.677639,0.575916,0.275745,0.213353,0.397981,0.499913,6.407320
min,39.000000,90.000000,60.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,10.726644
25%,48.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,23.875115
50%,53.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,26.395803
75%,58.000000,140.000000,90.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,30.371764
max,64.000000,180.000000,120.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,278.125000


In [197]:
# Carregamento da base de dados no cbrkit
db = cbrkit.loaders.dataframe(df)

In [198]:
df["bp_category"].unique()

array(['Normal', 'Hypertension Stage 1', 'Hypertension Stage 2',
       'Elevated'], dtype=object)

### Construção do modelo CBR

> Para mostrar mais de uma forma de construção de similaridade, a coluna com dados categoricos nominais foi também selecionada.

Com o `cbrkit`, devemos construir a estrutura do modelo no `cbrkit.sim.attribute_value`, definindo as funções de similaridade para cada atributo (se necessário) e/ou para os tipos de dados, de modo a generalizar variáveis de mesma tipagem. As similaridades podem ser feitas em relação a variáveis numéricas, textuais, coleções ou até mesmo através de uma taxonomia (hierarquia). É necessário também definir o agregador (`cbrkit.sim.aggregator(...)`), informando a métrica utilizada por ele

#### Uso do Modelo

Para construir o modelo de recuperação, usa-se o `cbrkit.retrieval.build`, passando a estrutura do modelo e o valor limite de resultados retornados. Através do `cbrkit.retrieval.apply`, é possível recuperar os casos similares

In [206]:
# Construindo modelo baseado em similaridades e com agregador baseado na média

bp_category_similarity = [
    ("Normal", "Elevated", 0.5), 
    ("Normal", "Hypertension Stage 1", 0.3),
    ("Normal", "Hypertension Stage 2", 0.1),
    ("Elevated", "Hypertension Stage 1", 0.5),
    ("Elevated", "Hypertension Stage 2", 0.3),
    ("Hypertension Stage 2", "Hypertension Stage 1", 0.5),
]
    
    

global_similarity = cbrkit.sim.attribute_value(
    attributes={
        "bp_category": cbrkit.sim.strings.table(bp_category_similarity,
                                                symmetric=True, default=0.0)
    },
    types={
        np.int64: cbrkit.sim.numbers.linear(max=9999),
        np.float64: cbrkit.sim.numbers.sigmoid(1,10)
    },
    aggregator=cbrkit.sim.aggregator("mean")
)

# Função para realização de queries
def run_query(query, limit = 5):
    retriever = cbrkit.retrieval.build(global_similarity, limit=limit)

    return cbrkit.retrieval.apply(db, query, retriever)

def print_results(results):
    for rank, (case_id, sim) in enumerate(results.similarities.items()):
        print(f"Rank {rank + 1}: Case {case_id + 1}")
        print(f"Case:\n{db[case_id]}")
        print(f"Global similarity: {sim.value:.3f}")
        print(f"Local similarities: {sim.by_attribute}")
        print("\n\n")

### Exemplos de Uso

Considerando uma pressão arterial levemente alta, colesterol e glucose acima do normal e fatores como fumo e bebida, é possível ver que os 4 resultados mais similares apontam pra um `cardio = 1`, indicando o risco/presença de doença cardiovascular

In [207]:
# Exemplo de query
query = pd.Series(
 {"age": 42,
 "ap_hi": 150,
 "ap_lo": 80,
 "cholesterol": 2,
 "gluc": 2,
 "smoke": 1,
 "alco": 1,
 "active": 0,
 "bmi": 28})

results = run_query(query, limit=4)

# Casos Similares Recuperados
print_results(results)

Rank 1: Case 6264
Case:
age                              45
ap_hi                           150
ap_lo                            80
cholesterol                       2
gluc                              1
smoke                             1
alco                              0
active                            1
cardio                            1
bmi                       27.681661
bp_category    Hypertension Stage 1
Name: 6263, dtype: object
Global similarity: 1.000
Local similarities: {'ap_hi': np.float64(1.0), 'active': np.float64(0.9998999899989999), 'bmi': np.float64(0.999968162906256), 'cholesterol': np.float64(1.0), 'smoke': np.float64(1.0), 'ap_lo': np.float64(1.0), 'gluc': np.float64(0.9998999899989999), 'alco': np.float64(0.9998999899989999), 'age': np.float64(0.9996999699969997)}



Rank 2: Case 5324
Case:
age                              44
ap_hi                           150
ap_lo                            80
cholesterol                       2
gluc                        

Considerando números mais saudáveis, como colesterol normal, não fumante, nem alcólico, pressão normal, mas ainda com alguns fatores de risco, com glucose alta, é possível ver que houve metade dos casos sem e metade com presença de doença cardiovascular

In [209]:
# Exemplo de query 2
query = pd.Series(
 {"age": 40,
 "ap_hi": 120,
 "ap_lo": 80,
 "cholesterol": 1,
 "gluc": 2,
 "smoke": 0,
 "alco": 0,
 "active": 0,
 "bmi": 25})

results = run_query(query, limit=4)

# Casos Similares Recuperados
print_results(results)

Rank 1: Case 9431
Case:
age                              40
ap_hi                           120
ap_lo                            80
cholesterol                       1
gluc                              1
smoke                             0
alco                              0
active                            0
cardio                            0
bmi                       25.503616
bp_category    Hypertension Stage 1
Name: 9430, dtype: object
Global similarity: 1.000
Local similarities: {'ap_hi': np.float64(1.0), 'active': np.float64(1.0), 'bmi': np.float64(0.9999496333930914), 'cholesterol': np.float64(1.0), 'smoke': np.float64(1.0), 'ap_lo': np.float64(1.0), 'gluc': np.float64(0.9998999899989999), 'alco': np.float64(1.0), 'age': np.float64(1.0)}



Rank 2: Case 1715
Case:
age                              40
ap_hi                           120
ap_lo                            80
cholesterol                       1
gluc                              1
smoke                             0
